In [1]:
# 라이브러리
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from scipy import stats


In [11]:
# 데이터 로딩
df = pd.read_csv('/content/Raw_data.csv')
# 결측비율 70% 이상 컬럼 삭제
df = df.drop(columns = ['merchant_profile_picture', 'has_urgency_banner', 'urgency_text'])
# rating_one_count ~ _five_count 는 분석에 사용할 컬럼이며, 결측값이 작아서 삭제
list = []
for i in df[df['rating_one_count'].isnull() == True].index:
  list.append(i)
df = df.drop(index = list)
# origin_country의 대부분이 'CN'이므로 해당값으로 치환
df['origin_country'] = df['origin_country'].fillna('CN')
# 변경가능한 사이즈는 종류가 105개로 각기 다른기준으로 구분되어있어 컬럼을 삭제
df = df.drop(columns = 'product_variation_size_id')
# 앞선 결측값제거로 인한 인덱스 초기화
df = df.reset_index().drop(columns = 'index')
# 상품확인결과 기타 일반 물건에 의한 색상 구분이 어려워 삭제
df = df.drop(index = [448, 618, 690, 830, 944, 988, 997, 1004, 1083, 1152, 1301, 1330, 1427, 1431, 1463])
# 유럽언어제목을 삭제한 후 영어제목을 title로 변경
df = df.drop(columns = 'title')
df.rename(columns={'title_orig':'title'},inplace=True)
# 소매가격과 실제가격의 차이만으로 의미를 찾을 수없다고 판단(더 많은 변수들과 복합적이기 때문)
df = df.drop(columns = ['retail_price'])
# 통화는 모두 EUR사용하기 때문에 삭제
df = df.drop(columns = ['currency_buyer'])
# 테마는 모두 여름이고, 크롤링 날짜 또한 같은날 진행하였기 때문에 삭제
df = df.drop(columns = ['theme', 'crawl_month'])
# URL은 모델에 사용할 수 있는 컬럼이 아니고 인사이트도 없기 때문에 제거
df = df.drop(columns = ['product_picture', 'product_url'])
# 판매자 이름은 판매자 id와 겹치므로 제거
df = df.drop(columns = ['merchant_name'])
# 필요없는 컬럼 삭제(merchant_info_subtitle,merchant_title : 불필요한 텍스트/ inventory_total : 사용하지 않을 결측값)
df = df.drop(columns = ['merchant_info_subtitle', 'inventory_total', 'merchant_title'])
# 중복값 처리
df = df.drop_duplicates(keep = 'first')
df = df.reset_index()
df = df.drop(columns = 'index')
# successful_sold컬럼으로 변경
sold_list = []
for i in df['units_sold']:
  if i >= 5000:
    sold_list.append(1)
  else:
    sold_list.append(0)
df['successful_sold'] = sold_list
df = df.drop(columns = ['units_sold'])
df = df.drop(columns = ['shipping_option_name'])
df.drop(columns=['shipping_is_express'],inplace=True)
# 배송범위수정
country_list = []

for i in df['countries_shipped_to']:
  if i > 40:
    country_list.append(1)
  else:
    country_list.append(0)

df['countries_shipped_to'] = country_list
df.rename(columns = {'countries_shipped_to':'shipped_range'}, inplace = True)
# 1513개의 데이터중 1474개가 중국이므로 중국과 아닌것으로 변환

origin_list = []

for i in df['origin_country']:
  if i == 'CN':
    origin_list.append(1)
  else:
    origin_list.append(0)

df.rename(columns={'origin_country':'made_in_china'},inplace=True)
df['made_in_china'] = origin_list
df['rating'] = round(df['rating'],1)
# 로그변환
df['rating_count'] = np.log1p(df['rating_count'])
# 배송비
list =[]
for i in df['shipping_option_price']:
  if i >= 5:
    list.append(5)
  else:
    list.append(i)

df['shipping_option_price'] = list
# 로그변환
df['merchant_rating_count'] = np.log1p(df['merchant_rating_count'])
inven_list = []
for i in df['product_variation_inventory']:
  if i == 50:
    inven_list.append(1)
  else:
    inven_list.append(0)
# 사용하지 않는 컬럼제거
df = df.drop(columns = ['title', 'rating_five_count', 'rating_four_count', 'rating_three_count', 'rating_two_count', 'rating_one_count', 'tags', 'product_color', 'merchant_id', 'product_id' , 'made_in_china'])

# 데이터 셋 분리 
feature = df.columns.drop('successful_sold')
target = 'successful_sold'

X_train, X_test, y_train, y_test = train_test_split(df[feature], df[target], test_size = 0.2, random_state = 42)
X_train['price'] = stats.zscore(X_train['price'])
X_test['price'] = stats.zscore(X_test['price'])



# 모델링
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# 파이프라인 생성 및 학습
pipe = make_pipeline(
    RandomForestClassifier(random_state = 42, n_jobs = -1,n_estimators=200, max_depth=10,min_samples_leaf= 4)
)

pipe.fit(X_train, y_train)
# 시스템
from scipy import stats

# 정보 받기
a,b,c,d,e,f,g,h,i,j,k,l,m,n = map(float,input("정보를 입력해 주세요(띄어쓰기)").split())
list = np.array([a,b,c,d,e,f,g,h,i,j,k,l,m,n])
list = np.round(list.astype('float32'),1)
list = list.reshape(1,14)
# 데이터프레임으로 이동
test_sample = pd.DataFrame(data = list, columns = ['price', 'uses_ad_boosts', 'rating', 'rating_count', 'badges_count',
      'badge_local_product', 'badge_product_quality', 'badge_fast_shipping',
      'product_variation_inventory', 'shipping_option_price', 'shipped_range',
      'merchant_rating_count', 'merchant_rating',
      'merchant_has_profile_picture'])


# 전처리
test_sample['rating'] = round(test_sample['rating'],1)
test_sample['product_variation_inventory'] = [1 if i == 50 else 0 for i in test_sample['product_variation_inventory']]
test_sample['shipping_option_price'] = [5 if i >=5 else i for i in test_sample['shipping_option_price']]
test_sample['merchant_rating'] = round(test_sample['merchant_rating'], 1)


# 판매 예측
test_pred = pipe.predict(test_sample)
print(f'판매 예측 결과는 다음과 같습니다.(0은 실패, 1은 성공) : {test_pred}')

정보를 입력해 주세요(띄어쓰기)8 1 3.15 6 0 0 0 0 50 2 41 12 3.1125 0
판매 예측 결과는 다음과 같습니다.(0은 실패, 1은 성공) : [0]
